In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
import os
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import plotly.express as px
import requests
from bs4 import BeautifulSoup

In [28]:
all_company_list = []
all_inn_list = []
def parse_rusprofile():
    company_list = []
    inn_list = []
    url = "https://www.rusprofile.ru/search-advanced"
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(60)
    companies = driver.find_elements(By.CSS_SELECTOR, "div.company-item")
    for company in companies:
        company_name = company.find_element(By.CSS_SELECTOR, "span").text
        company_list.append(company_name)
        try:
            #ищем <dt> с текстом "ИНН" и берем следующий за ним <dd>
            inn = company.find_element(By.XPATH, ".//div[@class='company-item-info']//dt[contains(text(), 'ИНН')]/following-sibling::dd").text
        except:
            inn = "ИНН не найден"
        inn_list.append(inn)
    driver.quit()
    return company_list, inn_list
#47.76.1
#из-за особенности сайта ставлю больше времени и ввожу критерии вручную

In [29]:
#чтобы достать больше компаний, ввожу разные критерии по времени
for i in range(2):
    company_list, inn_list = parse_rusprofile()
    all_company_list.extend(company_list)
    all_inn_list.extend(inn_list)

In [13]:
access_key = ""

In [121]:
#для проверки реального адреса пробиваю через 2gis (яндекс платный)
def find_shop(name):
    params = {
    #"itin": f"{itin}",
    "region_id": 32,  
    "locale": "ru_RU",  
    "key": access_key,  
    "page_size": 1, #количество результатов
    "radius": 1000,
    "q": f"{name}",
    "fields": "items.point,items.reviews,items.rubrics,items.full_address_name,items.description"
    }
    response = requests.get("https://catalog.api.2gis.com/3.0/items", params=params)
    if response.status_code == 200:
        data = response.json()
        organizations = data.get("result", {}).get("items", [])
        org_data = []
        #обработка списка
        for org in organizations:
            name = org.get("name", "Нет названия")
            address = org.get("full_address_name", "Нет адреса")
            #description = org.get("description", "Нет описания")
            categories = ", ".join([cat.get("name", "") for cat in org.get("rubrics", [])]) 
            reviews = org.get("reviews", {}).get("review_count", 0)  
            rating = org.get("reviews", {}).get("rating", 0)
            org_data.append({
                "Название": name,
                "Адрес": address,
                #"Описание": description,
                "Категории": categories,
                "Количество отзывов": reviews,
                "Рейтинг": rating
            })
        
    else:
        print(f"Ошибка: {response.status_code}")
    return org_data

In [122]:
all_org_data = []
for name in all_company_list:
    org_data = find_shop(name)
    all_org_data.extend(org_data)
df_shops = pd.DataFrame(all_org_data)
df_shops

,Название,Адрес,Описание,Категории,Количество отзывов,Рейтинг
0,"Ooo-zao.ru, юридическая фирма","Москва, проспект Маршала Жукова, 2",Нет описания,Регистрация / ликвидация предприятий,0,0.0
1,"Ритейл, фирма","Москва, Большая Татарская улица, 5 ст5",Нет описания,Средства гигиены,0,0.0
2,"Фл лизинг, компания","Москва, улица Мясницкая, 46 ст3",Нет описания,Лизинговые услуги,0,0.0
3,"Ooo-zao.ru, юридическая фирма","Москва, проспект Маршала Жукова, 2",Нет описания,Регистрация / ликвидация предприятий,0,0.0
4,Первый Дачный,"Москва, улица Ленинская Слобода, 19",Нет описания,"Семена / Посадочный материал, Средства защиты ...",4,5.0
...,...,...,...,...,...,...
95,"СК, центр лицензирования","Москва, Орликов переулок, 4",Нет описания,"Лицензирование, Помощь в оформлении членства СРО",10,4.7
96,"Цветы, пансионат для пожилых","Покровское, село Покровское, ст1",Нет описания,Дома престарелых,0,0.0
97,"Ooo-zao.ru, юридическая фирма","Москва, проспект Маршала Жукова, 2",Нет описания,Регистрация / ликвидация предприятий,0,0.0
98,"Fiola, студия флористики и дизайна","Москва, Большой бульвар, 40",Нет описания,"Доставка цветов, Цветы",0,0.0


In [116]:
words_to_remove = ['Дома', 'Регистрация', 'Лизинговые', 'Продуктовые', 'гигиены', 'Меховые', 'Новостройки', 'Лицензирование', 'мебель', 'проектирование', 'Торговые', 'Детская', 'Автоматизация', 'улиц', 'Жилищно-коммунальные', 'Гравировка', 'Школьная', 'Бани', 'отопления', 'супермаркеты', 'Обучение', 'Косметолог', 'Каркасно-тентовые']

#вручную удаляю ошибочно показанные компании
column_name = 'Категории'  
df_filtered = df_shops[~df_shops[column_name].str.contains('|'.join(words_to_remove), case=False, na=False)]
df_filtered.reset_index(inplace = True, drop=True)
df_filtered = df_filtered.drop([6, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 25, 26, 27, 29, 32, 34, 35, 36, 37, 38, 40, 41, 42, 43])
df_filtered.reset_index(inplace = True, drop=True)
df_filtered

,Название,Адрес,Описание,Категории,Телефон,Ссылка на 2GIS,Количество отзывов,Рейтинг
0,Первый Дачный,"Москва, улица Ленинская Слобода, 19",Нет описания,"Семена / Посадочный материал, Средства защиты ...",Нет телефона,Нет ссылки,4,5.0
1,Космас Флауэрс,"Москва, улица Саморы Машела, 5а",Нет описания,Доставка цветов,Нет телефона,Нет ссылки,0,0.0
2,"Флора, компания","Москва, Самокатная улица, 4а",Нет описания,Средства защиты растений / Удобрения,Нет телефона,Нет ссылки,0,0.0
3,"Гламелия, оптово-розничная компания","Москва, Бережковская набережная, 12а ст5",Нет описания,Цветы,Нет телефона,Нет ссылки,0,0.0
4,Любит не любит,"Москва, Кропоткинский переулок, 9",Нет описания,"Цветы, Доставка цветов",Нет телефона,Нет ссылки,0,0.0
5,"Семко, компания продажи семян, селекции и семе...","Москва, Рижский проезд, 3",Нет описания,"Семена / Посадочный материал, Средства защиты ...",Нет телефона,Нет ссылки,0,0.0
6,"Театр цветов, салон","Москва, улица Октябрьская, 38 к2",Нет описания,"Цветы, Доставка цветов, Сувениры, Товары для п...",Нет телефона,Нет ссылки,0,0.0
7,"Гифт Рум Рус, магазин цветов и подарков","Москва, Уланский переулок, 23",Нет описания,"Сувениры, Цветы, Услуги по упаковке подарков, ...",Нет телефона,Нет ссылки,0,0.0
8,"Агрофирма Партнер, фирменный магазин","Москва, 1-я Останкинская улица, 26",Нет описания,"Семена / Посадочный материал, Растениеводство",Нет телефона,Нет ссылки,1,1.0
9,"Гламелия, оптово-розничная компания","Москва, Бережковская набережная, 12а ст5",Нет описания,Цветы,Нет телефона,Нет ссылки,0,0.0


In [115]:
word_to_count = 'Цветы'
column_name = 'Категории' 
count = df_filtered[column_name].str.contains(word_to_count, case=False, na=False).sum()

print(f"Слово '{word_to_count}' встречается {count} раз.")

Слово 'Цветы' встречается 19 раз.


In [ ]:
#19 из 27 продают цветы
#цветы отличаются сезонностью и кратковременностью
#наши растения можно арендовать в любое время и они будут долго украшать дом/квартиру